# Setup

In [12]:
import esg

from utils.ownership import parse_ownership_data
from utils.regressors import parse_regressors

In [13]:
import seaborn as sns

sns.set(style="white",
        palette="pastel",
        color_codes=True,
        rc={'figure.figsize': (12, 9) })

In [14]:
import pandas as pd
import statsmodels.api as sm

# Importing data

In [15]:
import dotenv, os

dotenv.load_dotenv()

data_path = os.path.join("..", os.environ.get("DATA_PATH"))

esg_path = os.path.join(data_path, "ESG.xlsx")
sheet_name = os.environ.get("SHEET_NAME")

own_path = os.path.join(data_path, "ownership.csv")
reg_path = os.path.join(data_path, "controls.csv")

saliency_path = os.path.join(data_path, "saliency.csv")

## Covariates

In [16]:
raw = pd.read_csv(reg_path)
covariates = parse_regressors(raw)

## ESG data

In [17]:
raw = pd.read_excel(esg_path, sheet_name = sheet_name)
get_esg = esg.yearly_esg_maker(raw)

## Ownership data

In [18]:
ownership = parse_ownership_data(pd.read_csv(own_path))
investors = ownership.columns[-7:]

## Saliency data

In [19]:
saliency_df = pd.read_csv(saliency_path, index_col = [0, 1])

# Regression

In [28]:
def makeregressors(
    saliency_df, ownership, covariates, index, report):
    
    S = saliency_df.loc[report, index]
    companies = ownership.columns[-7:]
    X = ownership[companies]
    X.columns = [f"{company} ownership".lower() for company in companies]
    X.index = ownership['TargetFirms']

    for company in S.index:
        column_name = f"{company} {report.lower()}"
        X.loc[:, f"{column_name} interaction"] = X[f"{company} ownership"] * S[company] 
    
    X = sm.add_constant(X, prepend = False, has_constant='add')

    return X

In [32]:
covariates.index

Index(['FLWS-US', 'SRCE-US', 'DDD-US', 'III-GB', '06RLB3-E', '3PL-AU',
       '8341-JP', 'EGHT-US', 'MAERSK.B-DK', 'A2B-AU',
       ...
       'ZIOP-US', 'Z1P-AU', 'ZM-US', 'ZO1-DE', '3092-JP', 'ZS-US', '000063-CN',
       'ZUMZ-US', 'ZAG-AT', 'ZUO-US'],
      dtype='object', name='Firms', length=2615)

In [24]:
X = makeregressors(saliency_df, ownership, covariates, "s-index", "STEWARDSHIP_POLICY")

X

/Users/andreatitton/economics/nlp-financial-env/.venv/lib/python3.8/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,nn ownership,blackrock ownership,nbim ownership,generali ownership,jpmam ownership,apg ownership,dws ownership,blackrock stewardship_policy interaction,dws stewardship_policy interaction,generali stewardship_policy interaction,jpmam stewardship_policy interaction,nn stewardship_policy interaction,apg stewardship_policy interaction,nbim stewardship_policy interaction,const
TargetFirms,,,,,,,,,,,,,,,
1-800-FLOWERS.COM'A',0.00000,0.04961,0.00000,0.0,0.00027,0.0,0.00069,0.000870,3.824485e-06,0.0,0.000003,0.000000,0.0,0.000000,1.0
1STSOURCE,0.00000,0.04011,0.01231,0.0,0.00815,0.0,0.00008,0.000704,4.434186e-07,0.0,0.000089,0.000000,0.0,0.000081,1.0
3DSYSTEMS,0.00000,0.14012,0.00418,0.0,0.00000,0.0,0.00247,0.002458,1.369055e-05,0.0,0.000000,0.000000,0.0,0.000027,1.0
3IGROUP,0.00104,0.01047,0.02111,0.0,0.00405,0.0,0.00109,0.000184,6.041578e-06,0.0,0.000044,0.000007,0.0,0.000139,1.0
3PLEARNING,0.00000,0.00000,0.00000,0.0,0.00000,0.0,0.00000,0.000000,0.000000e+00,0.0,0.000000,0.000000,0.0,0.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZSCALER,0.00000,0.02182,0.00482,0.0,0.01880,0.0,0.00036,0.000383,1.995384e-06,0.0,0.000205,0.000000,0.0,0.000032,1.0
ZTE'A',0.00000,0.00106,0.00000,0.0,0.00000,0.0,0.00000,0.000019,0.000000e+00,0.0,0.000000,0.000000,0.0,0.000000,1.0
ZUMIEZ,0.00000,0.11717,0.00000,0.0,0.01949,0.0,0.00118,0.002055,6.540424e-06,0.0,0.000213,0.000000,0.0,0.000000,1.0


In [32]:
y = get_esg(2020).loc[X.index]['ESG Combined Score'] / 100

In [33]:
X

,nn ownership,blackrock ownership,nbim ownership,generali ownership,jpmam ownership,apg ownership,dws ownership,blackrock stewardship_policy interaction,dws stewardship_policy interaction,generali stewardship_policy interaction,jpmam stewardship_policy interaction,nn stewardship_policy interaction,apg stewardship_policy interaction,nbim stewardship_policy interaction,const
TargetFirms,,,,,,,,,,,,,,,
1-800-FLOWERS.COM'A',0.00000,0.04961,0.00000,0.0,0.00027,0.0,0.00069,0.000870,3.824485e-06,0.0,0.000003,0.000000,0.0,0.000000,1.0
1STSOURCE,0.00000,0.04011,0.01231,0.0,0.00815,0.0,0.00008,0.000704,4.434186e-07,0.0,0.000089,0.000000,0.0,0.000081,1.0
3DSYSTEMS,0.00000,0.14012,0.00418,0.0,0.00000,0.0,0.00247,0.002458,1.369055e-05,0.0,0.000000,0.000000,0.0,0.000027,1.0
3IGROUP,0.00104,0.01047,0.02111,0.0,0.00405,0.0,0.00109,0.000184,6.041578e-06,0.0,0.000044,0.000007,0.0,0.000139,1.0
3PLEARNING,0.00000,0.00000,0.00000,0.0,0.00000,0.0,0.00000,0.000000,0.000000e+00,0.0,0.000000,0.000000,0.0,0.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZSCALER,0.00000,0.02182,0.00482,0.0,0.01880,0.0,0.00036,0.000383,1.995384e-06,0.0,0.000205,0.000000,0.0,0.000032,1.0
ZTE'A',0.00000,0.00106,0.00000,0.0,0.00000,0.0,0.00000,0.000019,0.000000e+00,0.0,0.000000,0.000000,0.0,0.000000,1.0
ZUMIEZ,0.00000,0.11717,0.00000,0.0,0.01949,0.0,0.00118,0.002055,6.540424e-06,0.0,0.000213,0.000000,0.0,0.000000,1.0
